##### ~/.keras/keras.json

    {
        "image_dim_ordering": "th" | "tf",
        "image_data_format": "channels_first", | "image_data_format": "channels_last",
        "epsilon": 1e-07,
        "floatx": "float32",
        "backend": "theano" | "tensorflow"
    }
    
select parameters whether you run model on theano or tensorflow
in this notebook, model is run on tensorflow x python 3.6 

In [1]:
from __future__ import division, print_function

%matplotlib inline

import numpy as np
import matplotlib
import os, json
import matplotlib.pyplot as plt

from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers import Input
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image
from keras import applications
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
FILES_PATH = 'http://files.fast.ai/models/'; CLASS_FILE='imagenet_class_index.json'
# Keras' get_file() is a handy function that downloads files, and caches them for re-use later
fpath = get_file(CLASS_FILE, FILES_PATH+CLASS_FILE, cache_subdir='models')
with open(fpath) as f: class_dict = json.load(f)
# Convert dictionary with string indexes into an array
classes = [class_dict[str(i)][1] for i in range(len(class_dict))]

In [3]:
# Mean of each channel as provided by VGG researchers
#vgg_mean = np.array([123.68, 116.779, 103.939]).reshape((3,1,1)) #theano complient
vgg_mean = np.array([123.68, 116.779, 103.939]).reshape((1,1,3)) #tensorflow complient
#vgg_mean.shape

In [4]:
#preprocess data (rearrange RGB + normalize)
def vgg_preprocess(x):
    x = x - vgg_mean     # subtract mean
    #print(x)
    #print(x.shape)
    return x[:, ::-1]    # reverse axis bgr->rgb

In [5]:
#convolutional block definition
def ConvBlock(layers, model, filters):
    for i in range(layers): 
        model.add(ZeroPadding2D((1,1)))
        model.add(Conv2D(filters, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))
    
#fully connected block definition
def FCBlock(model):
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    

#define the VGG model architecture
def VGG_16(weights_path=None):
    model = Sequential()
#   model.add(Lambda(vgg_preprocess, input_shape=(3,224,224))) #theano complient
    model.add(Lambda(vgg_preprocess, input_shape=(224,224,3))) #tensorflow complient

    ConvBlock(2, model, 64)
    ConvBlock(2, model, 128)
    ConvBlock(3, model, 256)
    ConvBlock(3, model, 512)
    ConvBlock(3, model, 512)

    model.add(Flatten())
    FCBlock(model)
    FCBlock(model)
    model.add(Dense(1000, activation='softmax'))
            
    if weights_path:
        model.load_weights(weights_path)
    
    return model

In [6]:
# download pretrained model
#fpath = get_file('vgg16.h5', FILES_PATH+'vgg16.h5', cache_subdir='models')
fpath = get_file('vgg16_weights_tf_dim_ordering_tf_kernels.h5','https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels.h5', cache_subdir='models')

In [7]:
fpath_fc = get_file('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5','https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', cache_subdir='models')

In [9]:
#create the model like any python object
model = VGG_16(fpath)
#model_fc = VGG_16_fc(fpath_fc)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_2 (Lambda)            (None, 224, 224, 3)       0         
_________________________________________________________________
zero_padding2d_14 (ZeroPaddi (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_15 (ZeroPaddi (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_16 (ZeroPaddi (None, 114, 114, 64)      0         
__________

### fine tuning model for our converse-newbalance battle

In [10]:
model.pop()
for layer in model.layers: layer.trainable=False

In [11]:
model.add(Dense(2, activation='softmax'))

In [12]:
#Show it!
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_2 (Lambda)            (None, 224, 224, 3)       0         
_________________________________________________________________
zero_padding2d_14 (ZeroPaddi (None, 226, 226, 3)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
zero_padding2d_15 (ZeroPaddi (None, 226, 226, 64)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
zero_padding2d_16 (ZeroPaddi (None, 114, 114, 64)      0         
__________

In [ ]:
path = "W://vincent-data/datasets/convbattle/"
batch_size = 5

def get_batches(dirname, gen=image.ImageDataGenerator(), shuffle=True, 
                batch_size=batch_size, class_mode='categorical'):
    return gen.flow_from_directory(path+dirname, target_size=(224,224), 
                class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

batches = get_batches('train', batch_size=batch_size)
val_batches = get_batches('valid', batch_size=batch_size)
imgs,labels = next(batches)
val_imgs, val_labels = next(val_batches)

In [ ]:
batch_size = 64
epochs = 50

nb_train_samples=len(batches.filenames)
nb_validation_samples=len(val_batches.filenames)

In [ ]:
model.output.shape

In [ ]:
top_model = Sequential()
top_model.add(Flatten(input_shape=x.shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

In [ ]:
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    path+'train',
    target_size=(224,224),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    path+'valid',
    target_size=(224,224),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size, verbose=2)